In [3]:
import esco_utils as eu

/home/amir/api/venv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Create Token

In [4]:
API_TOKEN = eu.get_token()
client = eu.Client(API_TOKEN)

{'data': {'token': '12a1bad0-4cb1-464c-8c8e-344957672bef'}, 'message': 'successful', 'meta': {}}


## Connect to DB & get vacancies samples 

In [5]:
vac_conn, skill_conn = eu.load_skillLab_DB()

## tarin models 

In [47]:
eu.train_models(['en','de','nl','es','pt','fr','ar'])

{"id": "9f9ee158-7312-11ec-9cb7-0242ac120005", "model_name": "tfidf_knn", "lang": "en", "ngram_min": 1, "ngram_max": 4, "n_neighbors": 50, "title_imp": 10, "alt_title_imp": 10, "case_insensitive": true}
{"id": "a326fba8-7312-11ec-902e-0242ac120005", "model_name": "tfidf_knn", "lang": "de", "ngram_min": 1, "ngram_max": 4, "n_neighbors": 50, "title_imp": 10, "alt_title_imp": 10, "case_insensitive": true}
{"id": "a5219c56-7312-11ec-86da-0242ac120005", "model_name": "tfidf_knn", "lang": "nl", "ngram_min": 1, "ngram_max": 4, "n_neighbors": 50, "title_imp": 10, "alt_title_imp": 10, "case_insensitive": true}
{"id": "a772ec9e-7312-11ec-a25c-0242ac120005", "model_name": "tfidf_knn", "lang": "es", "ngram_min": 1, "ngram_max": 4, "n_neighbors": 50, "title_imp": 10, "alt_title_imp": 10, "case_insensitive": true}
{"id": "aa3b7b94-7312-11ec-9567-0242ac120005", "model_name": "tfidf_knn", "lang": "pt", "ngram_min": 1, "ngram_max": 4, "n_neighbors": 50, "title_imp": 10, "alt_title_imp": 10, "case_insen

## Get all projects 

In [9]:
all_projs = client.get_all_projects()['data']['projects']
name2id = {prj['project_name']:prj['project_id'] for prj in all_projs}
name2id

{'GB': '61dd90dbce271774889fbc02',
 'DE': '61dd9651527776b760a845eb',
 'FR': '61dd96af527776b760a84719',
 'NL': '61dd9701527776b760a84847',
 'EL': '61dd97b7527776b760a84975',
 'PT': '61dd97da527776b760a84977',
 'ES': '61dda542527776b760a84981',
 'Mexico': '61ddbde4527776b760a84aaf',
 'United Kingdom': '61ddbe23527776b760a84bdd',
 'France': '61ddbeb7527776b760a84d0b',
 'Brazil': '61ddc0fb527776b760a84e39',
 'Netherlands': '61ddc2de527776b760a84f67',
 'Germany': '61ddc34d527776b760a85095',
 'Argentina': '61ddc51b527776b760a851c3',
 'Saudi Arabia': '61ddc5d6527776b760a852e9',
 'In Saudi Arabia': '61ddc9a2527776b760a85410',
 'test Saudi Arabia': '61ddc9ea527776b760a85536',
 'test 2 Saudi Arabia': '61ddcbed527776b760a8565c',
 'test 3 Saudi Arabia': '61ddd1f5527776b760a85782',
 'test 4 Saudi Arabia': '61ddd739527776b760a858a8',
 'GBR': '61dde40a527776b760a85f9f',
 'DEU': '61dde42a527776b760a860cd',
 'NLD': '61dde4cb527776b760a861fb',
 'FRA': '61dde4dc527776b760a86329',
 'MEX': '61dde50b52777

## create model 

In [ ]:
import pycountry

alpha2name = {country.alpha_2:country.name for country in pycountry.countries}
alpha2alpha3 = {country.alpha_2:country.alpha_3 for country in pycountry.countries}

labels_per_task = 5
lang = 'en'

if lang=='en':
    country = 'GB'
elif lang=='es':
    country = 'MX'
elif lang=='pt':
    country = 'BR'
elif lang=='ar':
    country = 'SA'
else:
    country = lang.upper()
project_name = 'test ' + alpha2name[country] 


models = get_models()
models = pd.DataFrame(models['data'])
model_id = models[models.lang==lang].id.iloc[0]
print(f"model_id for {lang} = {model_id}")

classes = get_all_tags(lang,skill_conn)
icon_path = f"./flags/{country}.png"
response = client.upload_file(icon_path)
icon_id = response['data']['file_id']
print(response)
print(f"icon {icon_path} uploaded with id {icon_id}")

# add icon to project
project = Project(project_name, 
                  labels_per_task,
                  metadata={'lang':lang}, 
                  model_id=model_id, 
                  icon_id=icon_id)
if project_name in name2id.keys():
    project_id = name2id[project_name]
    client.edit_project(project,project_id=project_id)
else:
    response = client.create_project(project)
    project_id = response['data']['project_id']
print(f"project id = {project_id}")

client.edit_project(Project(icon_id=icon_id),project_id=project_id)

add_classes(classes, project_id)

if lang=='ar':
    vacc = pd.read_csv('bayt.csv').rename(columns={'JobDescription': 'description','meta_Title': 'title'})
else:
    vacc = sample_vacancy(country,vac_conn)
add_samples(vacc, project_id)


## only for testing 
print("add labelers", client.add_labelers(project_id, ["test@test.com",]))

# Add (real) labelers 

In [87]:
proj2labelers = {
    'SAU': ['a.elhayek@skilllab.io'],
    'DEU': ['s.schmidt@skilllab.io', 'c.bretgeld@skilllab.io', 'r.martens@skilllab.io'],
    'GBR': ['k.binhumam@skilllab.io', 'g.portik@skilllab.io', 'a.radic@skilllab.io'],
    'BRA': ['v.soares@skilllab.io'],
    'MEX': ['a.chiba@skilllab.io'],
    'NLD': ['l.weller@skilllab.io'],
}

for pname, labelers in proj2labelers.items():
    pid = name2id[pname]
    client.add_labelers(pid,labelers)


In [10]:
client.add_labelers(name2id["DEU"],["s.schmid+data@skilllab.io"])

{'data': {'active_labelers': ['test@test.com',
   's.schmidt@skilllab.io',
   'c.bretgeld@skilllab.io',
   'r.martens@skilllab.io',
   's.schmid+data@skilllab.io'],
  'deactivated_labelers': [],
  'project_id': '61dde42a527776b760a860cd',
  'token': '12a1bad0-4cb1-464c-8c8e-344957672bef'},
 'message': 'successful',
 'meta': {}}

## Cancell all tasks in all projects 

In [59]:
projects = client.get_all_projects()['data']['projects']
for project in projects:
    project_id = project['project_id']
    states = ['in-progress', 'pending']
    for state in states: 
        tasks = client.get_all_tasks(project_id=project_id,status=state)
        task_ids = [task['_id'] for task in tasks['data']['tasks']]
        [client.cancel_task(project_id=project_id,task_id=task_id) for task_id in task_ids]

## cancell labeler 

In [85]:
for pname,pid in name2id.items():
    client.cancel_labeler(project_id=pid,email='test@test.com')
    if len(pname)==3:
        client.add_labelers(pid,['test@test.com'])